In [ ]:
import pandas as pd
from datetime import datetime

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

import vald_api_utilities as vau
import vald_forceFrame_api as vffa

%load_ext autoreload
%autoreload 2

# Initialize organizational parameters
Vald should provide a "Client ID", "Client Secret", and a "Tenant ID". The "Tenant ID" is some what interchangable with the "Team ID" and can be acquired through the External Tenant API call.

In [ ]:
# Enter information. tenant ID is optional and can found using the an API call.
client_id = ''
client_secret = ''
tenantId = ''

### Initialize Vald API Class Function  
The class functions as a wrapper for the External Tenant and External Profile API calls.

In [ ]:
vald = vau.vald_api(client_id,client_secret,tenant_id=tenantId)

### Step 1 - Get Token
"get_token()" should always be your first api call as this gives you a temparoray access key to be used for all future calls

In [ ]:
vald.get_token()

#### OPTIONAL - "get_all_tenants()" or "get_tenant_info()"
Use "get_all_tenants()" to return a dictionary with all tenants.
Use "get_tenant_info()" to return a dictionary with additional inforamtion about specific tenant.

In [ ]:
vald.get_all_tenants()
vald.tenants

In [ ]:
# If not initializing class function with "tenant ID" then enter 
# tenant_id here, using commented example.
# vald.get_tenant_info(tenant_id=vald.tenants[0]['id'])

# If initialized class function with "Tenant ID" then use the following.
vald.get_tenant_info()
vald.tenant_info

### Step 2 - Get information on the categories and groups
Using the specified "Tenant ID", get information on the categories and groups for the "Tenant ID".

In [ ]:
# Generated dataframe containing ID and name of each category
vald.get_tenant_categories()
vald.categories_df

In [ ]:
# Generated dataframe of groups ID, name, and category ID. 
vald.get_tenant_groups()
vald.groups_df

### Step 3 - Get all profiles in a specific group.
Use "get_group_profiles()" to generate dataframe of all athlete profiles in a given group based on the category and group dataframe. It is important to generate both the group and category dateframes as both of these will used to identify the correct IDs.

In [ ]:
# categoru_name is used to get the category ID which is used to get the correct group ID.
# This insures that when there are groups' with the same name in different categories, 
# the correct group ID is found.

# Replace 'Baseball' (group name) with desired value.
vald.get_group_profiles('Baseball',category_name='Team')
vald.profile_df.head()

# Force Frame API Call
After gathering the tenant ID and profile IDs we can use this information to pull force decks results.

In [ ]:
# Initialize Force Decks class function with tenant ID and header from Vald class function.
forceFrame = vffa.forceFrame_api(vald.tenant_id,vald.header,region='USA')

### Step 1 - Get a collection of tests
Using "get_multiple_tests()", you need to include a start and stop date for the period you'd like to pull tests from. You can also provide a modified date and or a profile ID to filter on. If no modified data is given then the start date will be used for the modified date. If no profile ID is given then that will not be included in API call.

Start and stop date should be within 6 months of each other

In [ ]:
start_date = '30/08/2024' # date must be in "dd/mm/yyyy" format
stop_date = '30/10/2024' # date must be in "dd/mm/yyyy" format
profileID = vald.profile_df.loc[13,'profileId']

forceFrame.get_multiple_tests(start_date,stop_date,profileID=profileID)
forceFrame.tests_df.head()

### Step 2 - Get individual test results
Using "get_test_results()", a test ID is used to pull the results for each rep within a denoted test. The resulting dataframe contains a row for each result from each rep.

In [ ]:
testID = forceFrame.tests_df.loc[0,'testId']
forceFrame.get_test_results(testID)
forceFrame.test

### Step 3 - Get the force trace for a given test
Using "get_force_trace()" will return the force trace in a dataframe for a given test ID.

In [ ]:
forceFrame.get_force_trace(testID)
forceFrame.raw

In [ ]:
# To access raw force trace within API call response
forceFrame.raw['forces'].head()